# Mnist Mindspore

https://www.mindspore.cn/tutorials/en/r1.9/beginner/quick_start.html

In [1]:
import mindspore
from mindspore import nn
from mindspore import ops
from mindspore.dataset import vision, transforms
from mindspore.dataset import MnistDataset

In [3]:
!pip install download

In [4]:
#Download data from open datasets
from download import download

url = "https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/" \
      "notebook/datasets/MNIST_Data.zip"
path = download(url, "./", kind="zip", replace=True)


file_sizes: 100%|███████████████████████████| 10.8M/10.8M [00:20<00:00, 536kB/s]
Extracting zip file...
Successfully downloaded / unzipped to ./


In [5]:
train_dataset = MnistDataset('MNIST_Data/train')
test_dataset = MnistDataset('MNIST_Data/test')

In [6]:
print(train_dataset.get_col_names())

['image', 'label']


In [8]:
def datapipe(dataset, batch_size):
    image_transforms = [
        vision.Rescale(1.0 / 255.0, 0),
        vision.Normalize(mean=(0.1307,), std=(0.3081,)),
        vision.HWC2CHW()]

    label_transform = transforms.TypeCast(mindspore.int32)

    dataset = dataset.map(image_transforms, 'image')
    dataset = dataset.map(label_transform, 'label')
    dataset = dataset.batch(batch_size)
    return dataset

In [9]:
train_dataset = datapipe(train_dataset, 64)
test_dataset = datapipe(test_dataset, 64)

In [10]:
for image, label in test_dataset.create_tuple_iterator():
    print(f"Shape of image [N, C, H, W]: {image.shape} {image.dtype}")
    print(f"Shape of label: {label.shape} {label.dtype}")
    break

Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


In [11]:
for data in test_dataset.create_dict_iterator():
    print(f"Shape of image [N, C, H, W]: {data['image'].shape} {data['image'].dtype}")
    print(f"Shape of label: {data['label'].shape} {data['label'].dtype}")
    break

Shape of image [N, C, H, W]: (64, 1, 28, 28) Float32
Shape of label: (64,) Int32


## Criando a Rede Neural

In [12]:
# Define model
class Network(nn.Cell):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.dense_relu_sequential = nn.SequentialCell(
            nn.Dense(28*28, 512),
            nn.ReLU(),
            nn.Dense(512, 512),
            nn.ReLU(),
            nn.Dense(512, 10)
        )

    def construct(self, x):
        x = self.flatten(x)
        logits = self.dense_relu_sequential(x)
        return logits

model = Network()
print(model)

Network<
  (flatten): Flatten<>
  (dense_relu_sequential): SequentialCell<
    (0): Dense<input_channels=784, output_channels=512, has_bias=True>
    (1): ReLU<>
    (2): Dense<input_channels=512, output_channels=512, has_bias=True>
    (3): ReLU<>
    (4): Dense<input_channels=512, output_channels=10, has_bias=True>
    >
  >


## Treinando a Rede Neural

In [13]:
# Instantiate loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = nn.SGD(model.trainable_params(), 1e-2)

In [14]:
def train(model, dataset, loss_fn, optimizer):
    # Define forward function
    def forward_fn(data, label):
        logits = model(data)
        loss = loss_fn(logits, label)
        return loss, logits

    # Get gradient function
    grad_fn = ops.value_and_grad(forward_fn, None, optimizer.parameters, has_aux=True)

    # Define function of one-step training
    def train_step(data, label):
        (loss, _), grads = grad_fn(data, label)
        loss = ops.depend(loss, optimizer(grads))
        return loss

    size = dataset.get_dataset_size()
    model.set_train()
    for batch, (data, label) in enumerate(dataset.create_tuple_iterator()):
        loss = train_step(data, label)

        if batch % 100 == 0:
            loss, current = loss.asnumpy(), batch
            print(f"loss: {loss:>7f}  [{current:>3d}/{size:>3d}]")


In [15]:
def test(model, dataset, loss_fn):
    num_batches = dataset.get_dataset_size()
    model.set_train(False)
    total, test_loss, correct = 0, 0, 0
    for data, label in dataset.create_tuple_iterator():
        pred = model(data)
        total += len(data)
        test_loss += loss_fn(pred, label).asnumpy()
        correct += (pred.argmax(1) == label).asnumpy().sum()
    test_loss /= num_batches
    correct /= total
    print(f"Test: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [16]:
epochs = 3
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(model, train_dataset, loss_fn, optimizer)
    test(model, test_dataset, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.302964  [  0/938]
loss: 2.288135  [100/938]
loss: 2.273950  [200/938]
loss: 2.210056  [300/938]
loss: 1.969554  [400/938]
loss: 1.459011  [500/938]
loss: 0.963085  [600/938]
loss: 0.796074  [700/938]
loss: 0.625658  [800/938]


[WARNING] MD(320012,7f9ce97fa640,python):2023-01-24-00:07:12.792.105 [mindspore/ccsrc/minddata/dataset/engine/datasetops/batch_op.cc:136] operator()] Memory consumption is more than 80.6716%, which may cause oom error. Please reduce num_parallel_workers size / optimize 'per_batch_map' function / other python data preprocess function to reduce memory usage.


loss: 0.737398  [900/938]


[WARNING] MD(320012,7f9ceaffd640,python):2023-01-24-00:07:17.470.732 [mindspore/ccsrc/minddata/dataset/engine/datasetops/batch_op.cc:136] operator()] Memory consumption is more than 80.3619%, which may cause oom error. Please reduce num_parallel_workers size / optimize 'per_batch_map' function / other python data preprocess function to reduce memory usage.


Test: 
 Accuracy: 84.9%, Avg loss: 0.535465 

Epoch 2
-------------------------------
loss: 0.640206  [  0/938]
loss: 0.426942  [100/938]
loss: 0.681979  [200/938]
loss: 0.528917  [300/938]
loss: 0.389057  [400/938]
loss: 0.407892  [500/938]
loss: 0.477401  [600/938]
loss: 0.386485  [700/938]
loss: 0.424217  [800/938]


...

Test: 
 Accuracy: 89.9%, Avg loss: 0.343082 

Epoch 3
-------------------------------
loss: 0.261640  [  0/938]
loss: 0.426163  [100/938]
loss: 0.444421  [200/938]
loss: 0.361774  [300/938]
loss: 0.336551  [400/938]
loss: 0.278514  [500/938]
loss: 0.171455  [600/938]
loss: 0.242975  [700/938]
loss: 0.256046  [800/938]


[WARNING] MD(320012,7f9ce17fa640,python):2023-01-24-00:08:10.191.732 [mindspore/ccsrc/minddata/dataset/engine/datasetops/batch_op.cc:136] operator()] Memory consumption is more than 80.908%, which may cause oom error. Please reduce num_parallel_workers size / optimize 'per_batch_map' function / other python data preprocess function to reduce memory usage.


loss: 0.234700  [900/938]


[WARNING] MD(320012,7f9ceaffd640,python):2023-01-24-00:08:14.616.731 [mindspore/ccsrc/minddata/dataset/engine/datasetops/batch_op.cc:136] operator()] Memory consumption is more than 81.09%, which may cause oom error. Please reduce num_parallel_workers size / optimize 'per_batch_map' function / other python data preprocess function to reduce memory usage.


Test: 
 Accuracy: 91.7%, Avg loss: 0.287298 

Done!


## Salvando Checkpoint

In [17]:
# Save checkpoint
mindspore.save_checkpoint(model, "model.ckpt")
print("Saved Model to model.ckpt")

Saved Model to model.ckpt


## Carregando Checkpoint

In [18]:
# Instantiate a random initialized model
model = Network()
# Load checkpoint and load parameter to model
param_dict = mindspore.load_checkpoint("model.ckpt")
param_not_load = mindspore.load_param_into_net(model, param_dict)
print(param_not_load)

[]


In [19]:
model.set_train(False)
for data, label in test_dataset:
    pred = model(data)
    predicted = pred.argmax(1)
    print(f'Predicted: "{predicted[:10]}", Actual: "{label[:10]}"')
    break

Predicted: "[4 6 2 8 1 2 4 1 0 7]", Actual: "[4 6 2 8 1 2 4 1 0 7]"
